In [2]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold



In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train.head(5)

,id,scrape_id,last_scraped,name,description,picture_url,host_id,host_name,host_since,host_is_superhost,...,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,0,20230903194229,2023-09-04,Home in Torrance · 1 bedroom · 1 bed · 1 share...,"Beautifully designed decor, lighting and enter...",https://a0.muscache.com/pictures/0f019985-8295...,205539927,Leonardo,2018-07-27,f,...,363,2023-09-04,0,0,0,f,5,0,0,5
1,1,20230903194229,2023-09-04,Rental unit in Venice · ★4.74 · 2 bedrooms · 2...,The location is great - the apt is HUGH 1877 S...,https://a0.muscache.com/pictures/2d5c133d-a280...,7599550,Donald,2013-07-19,t,...,296,2023-09-04,81,24,2,f,3,3,0,0
2,2,20230903194229,2023-09-04,Home in Avalon · ★5.0 · 3 bedrooms · 4 beds · ...,"315 Eucalyptus Lower<br />Spacious, Remodeled ...",https://a0.muscache.com/pictures/prohost-api/H...,271118401,Catalina Vacations,2019-06-24,f,...,201,2023-09-04,3,2,0,t,113,113,0,0
3,3,20230903194229,2023-09-04,Home in Santa Clarita · 4 bedrooms · 5 beds · ...,Close to almost everything when you stay at th...,https://a0.muscache.com/pictures/miso/Hosting-...,429951162,Kellie,2021-11-01,f,...,364,2023-09-04,0,0,0,f,3,3,0,0
4,4,20230903194229,2023-09-04,Hotel in Los Angeles · ★New · 1 bedroom · 1 be...,Enjoy the thrill of staying in a hotel located...,https://a0.muscache.com/pictures/prohost-api/H...,501999278,RoomPicks,2023-02-20,f,...,269,2023-09-04,0,0,0,t,80,21,59,0


In [5]:
train.columns

Index(['id', 'scrape_id', 'last_scraped', 'name', 'description', 'picture_url',
       'host_id', 'host_name', 'host_since', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listi

In [6]:
X = train.iloc[:, 7:]
X_test = test.iloc[:, 7:]

In [7]:
X = X.drop(['host_since',"calendar_last_scraped", "latitude", "longitude"], axis=1)
X_test = X_test.drop(['host_since',"calendar_last_scraped", "latitude", "longitude"], axis=1)

In [8]:
X.dropna(inplace=True)
categorical_columns = X.select_dtypes(include=['object']).columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

X_test.dropna(inplace=True)
categorical_columns = X_test.select_dtypes(include=['object']).columns
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)

In [9]:
def train_multinomial_logistic_regression(data, target_column, test_size=0.2, random_state=42):
    """
    Trains a multinomial logistic regression classifier and evaluates its performance.

    Parameters:
    - X: Features (independent variables)
    - y: Target variable (dependent variable)
    - test_size: Proportion of the dataset to include in the test split (default is 0.2)
    - random_state: Seed used by the random number generator (default is 42)

    Returns:
    - classifier: Trained multinomial logistic regression classifier
    - accuracy: Accuracy of the classifier on the test set
    - conf_matrix: Confusion matrix
    - class_report: Classification report
    """

    X = data.drop(target_column, axis=1)
    y = data[target_column]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a multinomial logistic regression classifier
    classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

    # Train the classifier on the training data
    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test)

    # Evaluate the performance of the classifier
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Print the confusion matrix
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)

    # Print the classification report
    class_report = metrics.classification_report(y_test, y_pred)

    return classifier, accuracy, conf_matrix, class_report

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def train_random_forest(data, target_column, test, n_estimators=100, test_size=0.2, random_state=42):
    """
    Trains a Random Forest classifier and evaluates its performance.

    Parameters:
    - X: Features (independent variables)
    - y: Target variable (dependent variable)
    - n_estimators: The number of trees in the forest (default is 100)
    - test_size: Proportion of the dataset to include in the test split (default is 0.2)
    - random_state: Seed used by the random number generator (default is 42)

    Returns:
    - classifier: Trained Random Forest classifier
    - accuracy: Accuracy of the classifier on the test set
    - conf_matrix: Confusion matrix
    - class_report: Classification report
    """

    X = data.drop(target_column, axis=1)
    y = data[target_column]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a Random Forest classifier
    classifier = RandomForestClassifier(random_state=random_state)

    # Create stratified k-fold cross-validator
    stratified_kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)

    # Create GridSearchCV
    grid_search = GridSearchCV(classifier, param_grid, cv=stratified_kfold, scoring='accuracy')

    # Fit the GridSearchCV to the training data
    grid_search.fit(X_train, y_train)

    # Get the best classifier from the grid search
    best_classifier = grid_search.best_estimator_

    # Make predictions on the test set using the best classifier
    y_pred = best_classifier.predict(X_test)

    # Evaluate the performance of the best classifier
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Print the confusion matrix
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)

    # Print the classification report
    class_report = metrics.classification_report(y_test, y_pred)

    return best_classifier, accuracy, conf_matrix, class_report

classifier, accuracy, conf_matrix, class_report = train_random_forest(X, "price")
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Accuracy: 0.54
Confusion Matrix:
[[668  91   8  10   2   0]
 [172 218  70  90   7   1]
 [ 96 116 140 171  13   0]
 [ 58  58  71 285  50   5]
 [ 19  12  16 129 131  26]
 [ 15   0   2  22  44 162]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.65      0.86      0.74       779
         1.0       0.44      0.39      0.41       558
         2.0       0.46      0.26      0.33       536
         3.0       0.40      0.54      0.46       527
         4.0       0.53      0.39      0.45       333
         5.0       0.84      0.66      0.74       245

    accuracy                           0.54      2978
   macro avg       0.55      0.52      0.52      2978
weighted avg       0.53      0.54      0.52      2978



In [11]:
# train_multinomial_logistic_regression(X, "price")

/Users/michellesi/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/michellesi/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/michellesi/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(LogisticRegression(max_iter=1000, multi_class='multinomial'),
 0.17494963062458024,
 array([[  0,   0,  51, 728,   0,   0],
        [  0,   0,  62, 496,   0,   0],
        [  0,   0,  37, 499,   0,   0],
        [  0,   0,  43, 484,   0,   0],
        [  0,   0,  52, 281,   0,   0],
        [  0,   0,  24, 221,   0,   0]]),
 '              precision    recall  f1-score   support\n\n         0.0       0.00      0.00      0.00       779\n         1.0       0.00      0.00      0.00       558\n         2.0       0.14      0.07      0.09       536\n         3.0       0.18      0.92      0.30       527\n         4.0       0.00      0.00      0.00       333\n         5.0       0.00      0.00      0.00       245\n\n    accuracy                           0.17      2978\n   macro avg       0.05      0.16      0.07      2978\nweighted avg       0.06      0.17      0.07      2978\n')